In [1]:
import numpy
import os, ctypes
from scipy import integrate, LowLevelCallable
%run "integration_utils.py"
%run "q_solver.py"

In [3]:
y = numpy.matrix('1 1; 1 -1; -1 1; -1 -1', dtype=float)

lib = ctypes.CDLL(os.path.abspath('std_gaussian.so'))
lib.f.restype = ctypes.c_double
lib.f.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_double))
lib.g.restype = ctypes.c_double
lib.g.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_double), ctypes.c_void_p)
p1 = LowLevelCallable(lib.f)

In [ ]:
def g_w_multi_source(v, y):
    n = len(y)
    w = numpy.zeros(n)
    g = v.dot(q)
    for j in range(n):
        interval = gen_intervals(j, v, y)
        integral1 = integrate_over_intervals(interval, p1)
        user_data = ctypes.cast(ctypes.pointer((ctypes.c_double*2)(y[j,0], y[j,1])), ctypes.c_void_p)
        p2 = LowLevelCallable(lib.g, user_data)
        integral2 = integrate_over_intervals(interval, p2)
        w[j] = q[j] - integral1
        g += integral2 - v[j] * integral1
    return g, w

In [4]:
len(y)

4